# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 23 2022 9:00AM,247129,10,8563410,"Goodwin Biotechnology, Inc.",Released
1,Sep 23 2022 9:00AM,247128,19,60019238,"GUSA Granules USA, Inc.",Released
2,Sep 23 2022 8:54AM,247126,10,SNAP31955769,"Snap Medical Industries, LLC",Released
3,Sep 23 2022 8:54AM,247126,10,SNAP922221,"Snap Medical Industries, LLC",Released
4,Sep 23 2022 8:54AM,247126,10,SNAP922222,"Snap Medical Industries, LLC",Released
5,Sep 23 2022 8:54AM,247126,10,SNAP922223,"Snap Medical Industries, LLC",Released
6,Sep 23 2022 8:40AM,247124,10,8563413,Eywa Pharma Inc.,Released
7,Sep 23 2022 8:39AM,247123,10,0086003241,ISDIN Corporation,Released
8,Sep 23 2022 8:39AM,247123,10,0086003244,ISDIN Corporation,Released
9,Sep 23 2022 8:39AM,247123,10,0086003243,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,247123,Released,49
26,247124,Released,1
27,247126,Released,4
28,247128,Released,1
29,247129,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247123,NaN,NaN,49.0
247124,NaN,NaN,1.0
247126,NaN,NaN,4.0
247128,NaN,NaN,1.0
247129,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246891,0.0,1.0,0.0
246999,0.0,1.0,0.0
247000,0.0,1.0,0.0
247001,0.0,0.0,1.0
247031,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246891,0,1,0
246999,0,1,0
247000,0,1,0
247001,0,0,1
247031,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246891,0,1,0
1,246999,0,1,0
2,247000,0,1,0
3,247001,0,0,1
4,247031,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246891,,1,
1,246999,,1,
2,247000,,1,
3,247001,,,1
4,247031,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 23 2022 9:00AM,247129,10,"Goodwin Biotechnology, Inc."
1,Sep 23 2022 9:00AM,247128,19,"GUSA Granules USA, Inc."
2,Sep 23 2022 8:54AM,247126,10,"Snap Medical Industries, LLC"
6,Sep 23 2022 8:40AM,247124,10,Eywa Pharma Inc.
7,Sep 23 2022 8:39AM,247123,10,ISDIN Corporation
56,Sep 23 2022 8:30AM,247122,10,ISDIN Corporation
68,Sep 23 2022 8:30AM,247109,10,ISDIN Corporation
70,Sep 23 2022 8:30AM,247067,10,Eywa Pharma Inc.
71,Sep 23 2022 8:30AM,247104,10,ISDIN Corporation
82,Sep 23 2022 8:30AM,247105,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 23 2022 9:00AM,247129,10,"Goodwin Biotechnology, Inc.",,,1
1,Sep 23 2022 9:00AM,247128,19,"GUSA Granules USA, Inc.",,,1
2,Sep 23 2022 8:54AM,247126,10,"Snap Medical Industries, LLC",,,4
3,Sep 23 2022 8:40AM,247124,10,Eywa Pharma Inc.,,,1
4,Sep 23 2022 8:39AM,247123,10,ISDIN Corporation,,,49
5,Sep 23 2022 8:30AM,247122,10,ISDIN Corporation,,,12
6,Sep 23 2022 8:30AM,247109,10,ISDIN Corporation,,,2
7,Sep 23 2022 8:30AM,247067,10,Eywa Pharma Inc.,,,1
8,Sep 23 2022 8:30AM,247104,10,ISDIN Corporation,,,11
9,Sep 23 2022 8:30AM,247105,10,ISDIN Corporation,,,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 9:00AM,247129,10,"Goodwin Biotechnology, Inc.",1,,
1,Sep 23 2022 9:00AM,247128,19,"GUSA Granules USA, Inc.",1,,
2,Sep 23 2022 8:54AM,247126,10,"Snap Medical Industries, LLC",4,,
3,Sep 23 2022 8:40AM,247124,10,Eywa Pharma Inc.,1,,
4,Sep 23 2022 8:39AM,247123,10,ISDIN Corporation,49,,
5,Sep 23 2022 8:30AM,247122,10,ISDIN Corporation,12,,
6,Sep 23 2022 8:30AM,247109,10,ISDIN Corporation,2,,
7,Sep 23 2022 8:30AM,247067,10,Eywa Pharma Inc.,1,,
8,Sep 23 2022 8:30AM,247104,10,ISDIN Corporation,11,,
9,Sep 23 2022 8:30AM,247105,10,ISDIN Corporation,4,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 9:00AM,247129,10,"Goodwin Biotechnology, Inc.",1,,
1,Sep 23 2022 9:00AM,247128,19,"GUSA Granules USA, Inc.",1,,
2,Sep 23 2022 8:54AM,247126,10,"Snap Medical Industries, LLC",4,,
3,Sep 23 2022 8:40AM,247124,10,Eywa Pharma Inc.,1,,
4,Sep 23 2022 8:39AM,247123,10,ISDIN Corporation,49,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 9:00AM,247129,10,"Goodwin Biotechnology, Inc.",1.0,NaN,NaN
1,Sep 23 2022 9:00AM,247128,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
2,Sep 23 2022 8:54AM,247126,10,"Snap Medical Industries, LLC",4.0,NaN,NaN
3,Sep 23 2022 8:40AM,247124,10,Eywa Pharma Inc.,1.0,NaN,NaN
4,Sep 23 2022 8:39AM,247123,10,ISDIN Corporation,49.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 9:00AM,247129,10,"Goodwin Biotechnology, Inc.",1.0,0.0,0.0
1,Sep 23 2022 9:00AM,247128,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
2,Sep 23 2022 8:54AM,247126,10,"Snap Medical Industries, LLC",4.0,0.0,0.0
3,Sep 23 2022 8:40AM,247124,10,Eywa Pharma Inc.,1.0,0.0,0.0
4,Sep 23 2022 8:39AM,247123,10,ISDIN Corporation,49.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2965319,102.0,0.0,5.0
16,247107,1.0,0.0,0.0
19,1235517,15.0,2.0,0.0
20,741199,66.0,15.0,0.0
21,1235006,2.0,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2965319,102.0,0.0,5.0
1,16,247107,1.0,0.0,0.0
2,19,1235517,15.0,2.0,0.0
3,20,741199,66.0,15.0,0.0
4,21,1235006,2.0,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,102.0,0.0,5.0
1,16,1.0,0.0,0.0
2,19,15.0,2.0,0.0
3,20,66.0,15.0,0.0
4,21,2.0,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,102.0
1,16,Released,1.0
2,19,Released,15.0
3,20,Released,66.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20,21
Status,,,,,
Completed,5.0,0.0,0.0,0.0,0.0
Executing,0.0,0.0,2.0,15.0,3.0
Released,102.0,1.0,15.0,66.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20,21
0,Completed,5.0,0.0,0.0,0.0,0.0
1,Executing,0.0,0.0,2.0,15.0,3.0
2,Released,102.0,1.0,15.0,66.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20,21
0,Completed,5.0,0.0,0.0,0.0,0.0
1,Executing,0.0,0.0,2.0,15.0,3.0
2,Released,102.0,1.0,15.0,66.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()